<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import torch
import numpy as np
import pandas as pd

In [4]:
!pip install gensim

In [5]:
import gensim.downloader as api

model = api.load('glove-twitter-25')
# model.most_similar('awesome', topn=5).type()

[====================================--------------] 73.8% 77.4/104.8MB downloaded


In [6]:
def get_synonyms(word, n):
  try:
    words_list=model.most_similar(word, topn=n)
    return words_list  
  except Exception as e:
    return None
  

In [7]:
get_synonyms('awesome',3)

[('amazing', 0.9687871932983398),
 ('best', 0.9600659608840942),
 ('fun', 0.9331520795822144)]

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
import random
from random import sample
random.seed(17)
def synonym_replacement(text, n):
    text=str(text)
    text = text.split()

    random_word_list = list(set([word for word in text if word not in stopwords.words('english')]))
    random.shuffle(random_word_list)
    random_word_list = [word for word in random_word_list if get_synonyms(word,1)]
    try:
      raplace_words=sample(random_word_list,n)
    except Exception as e:
      raplace_words=random_word_list
    texts=[]
    for random_word in raplace_words:
        synonyms_word_list = get_synonyms(random_word,3)
        if synonyms_word_list:
            # synonym = random.choice(synonyms_word_list)
            synonym = synonyms_word_list[0]
            new_words = [synonym[0] if word == random_word else word for word in text]
        texts.append(' '.join(new_words))

    return texts

In [60]:
synonym_replacement("She should ask a few native Americans what their take on this is", 3)

['She should ask a few native Americans what their give on this is',
 'She should whats a few native Americans what their take on this is',
 'She should ask a few african Americans what their take on this is']

In [61]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')
df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [62]:

possible_labels=df.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13240 entries, 86426 to 46552
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      13240 non-null  object
 1   subtask_a  13240 non-null  object
 2   subtask_b  4400 non-null   object
 3   subtask_c  3876 non-null   object
dtypes: object(4)
memory usage: 517.2+ KB


In [118]:
import pandas as pd
df2 = pd.DataFrame({
    'id':[],
    'tweet':[],
    'subtask_a':[],
})
df2 = df2.set_index('id')

In [129]:
for index, row in df.iterrows():
    tweets=synonym_replacement(row['tweet'],3)    
    i=0
    for tweet in tweets:

      id=int(index*10+i+1)# 输出每行的索引值
      df2=df2.append([{'id':id,'tweet':tweets[i],'subtask_a':row['subtask_a']}],ignore_index=False)
      i+=1

In [132]:
df2.head()

,tweet,subtask_a
id,,
NaN,@USER She should ask a few african Americans w...,OFF
NaN,@USER She should ask a few native Americans wh...,OFF
NaN,@USER She should whats a few native Americans ...,OFF
864261.0,@USER She should ask a few native Americans wh...,OFF
864262.0,@USER She should whats a few native Americans ...,OFF


In [131]:
df2 = df2.set_index('id')

In [133]:
df2.to_csv('1.csv')

,tweet,subtask_a,id
0,@USER She should ask a few african Americans w...,0.0,0.0


In [123]:
df2

,tweet,subtask_a
id,,
864261,@USER She should ask a few african Americans w...,OFF
864262,@USER She should ask a few native Americans wh...,OFF
864263,@USER She should whats a few native Americans ...,OFF


,tweet,subtask_a
id,,
0.0,@USER She should ask a few native Americans wh...,[a]
1.0,@USER She should ask a few native Americans wh...,[a]
2.0,@USER She should ask a few african Americans w...,[a]
864261.0,@USER She should ask a few native Americans wh...,OFF
864262.0,@USER She should ask a few african Americans w...,OFF
864263.0,@USER She should whats a few native Americans ...,OFF
